In [4]:
import pyaudio
import wave

w = wave.open("/usr/share/sounds/alsa/Side_Left.wav", "rb")
data = w.readframes(w.getnframes())

p = pyaudio.PyAudio()
stream = p.open(format=p.get_format_from_width(2), 
               channels=1,
               rate=48000,
               output=True)

stream.write(data)

stream.stop_stream()
stream.close()

stream = p.open(format=p.get_format_from_width(w.getsampwidth()),
               channels=w.getnchannels(),
               rate=w.getframerate(),
               output=True)
w.rewind()

data = w.readframes(1024)
while len(data)>0:
    stream.write(data)
    data=w.readframes(1024)

w.close()
stream.close()
p.terminate()

In [5]:
import pyaudio
import wave
import time

def callback(in_data, frame_count, time_info, status):
    data=w.readframes(frame_count)
    return(data, pyaudio.paContinue)

w = wave.open("/usr/share/sounds/alsa/Side_Left.wav", "rb")
p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(w.getsampwidth()),
               channels=w.getnchannels(), 
               rate=w.getframerate(),
               output=True, 
               stream_callback=callback)

stream.start_stream()

while stream.is_active():
    print("main work...")
    time.sleep(0.1)
    
stream.stop_stream()
stream.close()
p.terminate()

main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...
main work...


In [13]:
import pyaudio
import numpy as np

volume = 0.5
fs = 48000
duration=1.0
f=440.0

data=(np.sin(2*np.pi*np.arange(fs*duration)*f/fs).astype(np.float32))

p=pyaudio.PyAudio()
stream=p.open(format=pyaudio.paFloat32, channels=1, rate=fs, output=True)
stream.write(volume*data)

stream.stop_stream()
stream.close()
p.terminate()

In [39]:
import pyaudio
import numpy as np

# 여기서 self는 자기자신을 가르킴 그래서 def 함수를 여러개 사용해서 
# 하나는 1 하나는 3의 값을 입력하면 각 객체마다 다른 값을 가질 수 있는거임! 

# 객체가 생성할때 무조건 생성자가 호출되어야 함! 그래서 우리가 생성자라고 파이썬의 생성자=__init__
# 생성자가 self.valume = volume
# __init__은 생성자이지만 또다른 기능은 얘가 있어야 패키지로서의 기능도 할수 잇음! 오 

# java는 생성자가 클래스 이름과 동일함(cpp도!)
# 파이썬은 생성자의 이름이 __init__임 
# 생성자는 입력 받을 수 있음, 입력값 없어도 됨 디폴트 값, return 값이 없음(출력이 없음) 
# 생성자는 객체를 초기화 하는 역할 

class Tone:
    def __init__(self, volume=.5, rate=48000, channels=1): 
        self.volume = volume   # 이런 부분은 함수 정의~ 함수 정의, 호출, 선언 잘 기억해두자! 
        self.rate = rate
        self.channels = channels
        self.p=pyaudio.PyAudio()
        self.stream = self.p.open(format=pyaudio.paFloat32,
                                  channels=self.channels, rate=self.rate, output=True)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.stop()  # 함수 호출

    def stop(self):  # 파이썬은 함수의 정의를 뒤에 해도 됨! 오.. 편리 
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

    def play(self, octave, note, duration):
        f = 2**(octave) * 55 * 2**(((note) - 10)/12) 
        sample = (np.sin(2* np.pi* np.arange(self.rate * duration) * f / self.rate)).astype(np.float32)
        self.stream.write(self.volume * sample)

with Tone() as tone:
    for n in [1, 3, 5, 7, 8, 10, 12]:
        tone.play(3, n, 4)

KeyboardInterrupt: 